In [1]:
import numpy as np
import ipywidgets as widgets
from ipyleaflet import Map, Marker, AwesomeIcon
from IPython.display import display

Yield=None

def store_value(button):
    global Yield
    Yield=yield_input.value
    
# Function to calculate distance between two coordinates
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 111320 / 0.9144  # Conversion from degrees to yards
    return distance

# Function to add a new ship icon
def add_ship(ship_color, ship_type, initial_location):
    while True:
        # Generate random location within the map boundaries
        random_location = [
            main_icon.location[0] + np.random.uniform(-0.01, 0.01),
            main_icon.location[1] + np.random.uniform(-0.01, 0.01)
        ]

        # Check distance from existing icons
        too_close = False
        for icon in secondary_icons:
            distance = calculate_distance(icon.location, random_location)
            if distance < 100:
                too_close = True
                break

        if not too_close:
            break

    new_icon = Marker(
        location=random_location,
        draggable=True,
        icon=AwesomeIcon(name='circle', marker_color=ship_color, icon_color='white')
    )
    secondary_icons.append(new_icon)
    secondary_ship_types[new_icon] = ship_type
    m.add_layer(new_icon)

# Function to handle main icon movement
def on_main_icon_move(event, main_icon):
    main_icon.location = event['new']

# Function to finalize ship location and calculate distances
def finalize_ship_location(button):
    global distances
    # Move main icon to average position of 100 random movements within 50 yards of the original position
    main_icon_location = main_icon.location
    print(f"{Yield}")
    for _ in range(100):
        random_location = [
            main_icon_location[0] + np.random.uniform(-0.0005, 0.0005),
            main_icon_location[1] + np.random.uniform(-0.0005, 0.0005)
        ]
        distance = calculate_distance(main_icon_location, random_location)
        
        if distance <= 50:
            main_icon_location = random_location

    main_icon.location = main_icon_location

    # Calculate distances from main icon to all secondary icons
    distances = []
    distances1 = []
    for icon in secondary_icons:
        distance = calculate_distance(main_icon.location, icon.location)
        distances.append(distance)
        distances1.append((secondary_ship_types[icon], distance))

    #Display distances in the text widget1
    distances_text.value = "\n".join(f"{ship}: {distance:.2f} yards" for ship, distance in distances1)

# Function to handle map change
def on_map_change(change):
    if change['name'] == 'center':
        clear_ships()
        main_icon.location = change['new']

# Define initial locations for different maps
initial_locations = {
    "Pearl Harbor": [21.3445, -157.9746],
    "Norfolk Naval Station, Virginia": [36.9375, -76.3186],
    "Middle of the Atlantic Ocean": [27.4076, -40.3018],
    "Middle of the Pacific Ocean": [27.3897, 166.5527]
}

# Create map selection dropdown widget
map_selection_dropdown = widgets.Dropdown(
    options=list(initial_locations.keys()),
    description='Select Map:',
    disabled=False
)

# Create main map
m = Map(center=initial_locations[map_selection_dropdown.value], zoom=15, layout=dict(height='600px'))

# Main icon
main_icon = Marker(
    location=initial_locations[map_selection_dropdown.value],
    draggable=True,
    icon=AwesomeIcon(name='circle', marker_color='red', icon_color='white')
)

# Initialize secondary icons list and ship types dictionary
secondary_icons = []
secondary_ship_types = {}

# Event handler for main icon movement
main_icon.observe(lambda event: on_main_icon_move(event, main_icon), names='location')

# Event handler for map change
m.observe(on_map_change)

# Event handler for map selection change
def on_map_selection_change(change):
    m.center = initial_locations[change['new']]
    clear_ships()
    main_icon.location = initial_locations[change['new']]

# Bind event handler to map selection dropdown
map_selection_dropdown.observe(on_map_selection_change, names='value')

# Add main icon to the map
m.add_layer(main_icon)

# Display the map selection dropdown
display(map_selection_dropdown)

yield_input = widgets.FloatText(description='Yield:')
yieldbutton = widgets.Button(description="Enter")
yieldbutton.on_click(store_value)
# Display the KT input
display(yield_input)
display(yieldbutton)

# Display the map
display(m)

# Event handler to add a carrier icon
def add_carrier(button):
    add_ship('blue', 'Carrier', m.center)

# Event handler to add a cruiser icon
def add_cruiser(button):
    add_ship('green', 'Cruiser', m.center)

# Event handler to add a destroyer icon
def add_destroyer(button):
    add_ship('black', 'Destroyer', m.center)

# Event handler to add a supply ship icon
def add_supply_ship(button):
    add_ship('white', 'Supply Ship', m.center)
    
    
def clear_ships(button):
    for icon in secondary_icons:
        m.remove_layer(icon)
    secondary_icons.clear()

# Add clear button
clear_button = widgets.Button(description="Clear Ships")
clear_button.on_click(clear_ships)

# Add clear button to the display
display(clear_button)

# Add secondary icon buttons
add_carrier_button = widgets.Button(description="Add Carrier")
add_carrier_button.on_click(add_carrier)
add_cruiser_button = widgets.Button(description="Add Cruiser")
add_cruiser_button.on_click(add_cruiser)
add_destroyer_button = widgets.Button(description="Add Destroyer")
add_destroyer_button.on_click(add_destroyer)
add_supply_ship_button = widgets.Button(description="Add Supply Ship")
add_supply_ship_button.on_click(add_supply_ship)

# Display buttons
ship_buttons = widgets.HBox([add_carrier_button, add_cruiser_button, add_destroyer_button, add_supply_ship_button])
display(ship_buttons)
# Display finalize ship location button
finalize_button = widgets.Button(description="Finalize Ship Location")
finalize_button.on_click(finalize_ship_location)
display(finalize_button)

# Text widget to display distances
distances_text = widgets.Textarea(layout={'height': '100px', 'width': '400px'})
display(distances_text)



Dropdown(description='Select Map:', options=('Pearl Harbor', 'Norfolk Naval Station, Virginia', 'Middle of the…

FloatText(value=0.0, description='Yield:')

Button(description='Enter', style=ButtonStyle())

Map(center=[21.3445, -157.9746], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

Button(description='Clear Ships', style=ButtonStyle())

Button(description='Finalize Ship Location', style=ButtonStyle())

Textarea(value='', layout=Layout(height='100px', width='400px'))

1.0
1.0


NameError: name 'distances1' is not defined